# Pesquisando por iniciativas/projetos com Dados Abertos Governamentais no Github

In [80]:
import requests
import pandas as pd
import time

## Mapeamento das principais bibliotecas

- [API da Câmara dos deputados](https://dadosabertos.camara.leg.br/swagger/api.html)

In [5]:
search_strings_libraries = []

Devido a estrutura de pesquisa no github as palavras chaves utilizadas para o 
news não foram tão eficiente, visto que preciso adicionar palavras de busca ou 
de forma conter uma das palavras ou conter todas as palavras na sequencia especificada.

Sendo assim:
- '"projeto" and "dados governo"',
- '"projeto" and "dados governamentais"',
- '"projeto" and "monitora" and "dados" and "governo"'

Não foram buscas efetivas, porém as palavras chaves inicias sim.

In [126]:
search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

'consumir' e 'publicação' não retornaram resultados e por isso foram retirados a fim de adicionar palavras que retornem resultados melhores
- 'consumir dados abertos do governo'
- 'consumir dados abertos governamentais'
- 'publicação de dados abertos do governo',
- 'publicação de dados governamentais',

In [121]:
sort = '&sort=stars&order=desc'
url_base = 'https://api.github.com/search/repositories?q='
credentials=('lorenaps','')

Verificando o limite de requisições

In [113]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1573085971},
 'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1573085971},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1573085971},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1573085971},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1573082431}}}

In [41]:
colunas=[
        'id',
        'full_name',
        'description',
        'owner_type', 
        'owner_url',
        'owner_html_url',
        'html_url',
        'url',
        'fork',
        'created_at',
        'updated_at',
        'size',
        'stargazers_count',
        'language',
        'has_issues',
        'has_wiki',
        'forks_count',
        'forks',
        'open_issues',
        'watchers',
        'timestamp_extract'
    ]


In [12]:
def add_resultado(item):
    df = pd.DataFrame([[
                        item.get('id'),
                        item.get('full_name', None),
                        item.get('description', None),      
                        item.get('owner').get('type', None),
                        item.get('owner').get('url', None),
                        item.get('owner').get('html_url', None),
                        item.get('html_url', None),
                        item.get('url', None),
                        item.get('fork', None),
                        item.get('created_at', None),
                        item.get('updated_at', None),
                        item.get('size', None),
                        item.get('stargazers_count', None),
                        item.get('language', None),
                        item.get('has_issues', None),
                        item.get('has_wiki', None),
                        item.get('forks_count', None),
                        item.get('forks', None),
                        item.get('open_issues', None),
                        item.get('watchers', None),
                        str(time.time()).split('.')[0]]], columns=colunas)

    return df    

In [13]:
def extrair_dados(data, resultados):
    for item in data.get('items', None):
        resultados = pd.concat([resultados, add_resultado(item)], ignore_index=True, sort=False)
        
    return resultados

Verificando limitação de extração de dados da API

In [14]:
mais_de_mil = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(mais_de_mil, auth=credentials)
t.json()

{'documentation_url': 'https://developer.github.com/v3/search/',
 'message': 'Only the first 1000 search results are available'}

In [127]:
resultados = pd.DataFrame(columns=['id',
                                    'full_name',
                                    'description',
                                    'owner_type', 
                                    'owner_url',
                                    'owner_html_url',
                                    'html_url',
                                    'url',
                                    'fork',
                                    'created_at',
                                    'updated_at',
                                    'size',
                                    'stargazers_count',
                                    'language',
                                    'has_issues',
                                    'has_wiki',
                                    'forks_count',
                                    'forks',
                                    'open_issues',
                                    'watchers',
                                    'timestamp_extract',
                                    'commits',
                                    'readme',
                                    'contributors',]) 

In [128]:
def percorrendo_paginas(url, resultados):
    
    print('Primeira requisição')
    
    results = requests.get(url, auth=credentials)    
    data = dict(results.json())
    total = data.get('total_count', None)
        
    print("Foram encontrados {0} resultados".format(total))

    resultados = extrair_dados(data, resultados)
    
    iteracoes = total // 30 
    
    for iteracao in range(0, iteracoes):        
        header = dict(results.links)
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
            
            print("\nNext url: {0}".format(next_url))
            print(">>>>>>>> Consumindo página: {0}".format(iteracao+2))
            
            results = requests.get(next_url, auth=credentials)
            data = dict(results.json())
            resultados = extrair_dados(data, resultados)
        
    return resultados

In [129]:
for string in search_strings:

    print("\nExtraindo repositórios para a string: '{0}'".format(string))

    url = url_base + string + sort
    resultados = percorrendo_paginas(url, resultados)


Extraindo repositórios para a string: 'dados abertos'
Primeira requisição
Foram encontrados 370 resultados

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=2
>>>>>>>> Consumindo página: 2

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=3
>>>>>>>> Consumindo página: 3

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=4
>>>>>>>> Consumindo página: 4

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=5
>>>>>>>> Consumindo página: 5

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=6
>>>>>>>> Consumindo página: 6

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=7
>>>>>>>> Consumindo página: 7

Next url: https://api.github.com/search/repositories?q=dados+abertos&sort=stars&order=desc&page=8
>>>>>>>> Consumindo

In [130]:
resultados.describe()

,id,full_name,description,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,...,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,readme,contributors
count,570,570,537,570,570,570,570,570,570,570,...,570,570,570,570,570,570,570,0.0,0.0,0.0
unique,424,424,389,2,366,366,424,424,1,424,...,2,2,16,16,14,25,29,0.0,0.0,0.0
top,195294598,prodest/ckanext-data_es_theme,Plugin/Tema do Portal de Dados Abertos do Gove...,User,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/thenets/ckanext-dadosabertos,https://api.github.com/repos/prodest/ckanext-d...,False,2019-07-04T19:52:41Z,...,True,True,0,0,0,0,1573082968,NaN,NaN,NaN
freq,6,6,6,461,16,16,6,6,570,6,...,555,554,423,423,487,377,30,NaN,NaN,NaN


In [131]:
resultados.head(5)

,id,full_name,description,owner_type,owner_url,owner_html_url,html_url,url,fork,created_at,...,has_issues,has_wiki,forks_count,forks,open_issues,watchers,timestamp_extract,commits,readme,contributors
0,29256552,CamaraDosDeputados/dados-abertos,Repositório do serviço de Dados Abertos da Câm...,Organization,https://api.github.com/users/CamaraDosDeputados,https://github.com/CamaraDosDeputados,https://github.com/CamaraDosDeputados/dados-ab...,https://api.github.com/repos/CamaraDosDeputado...,False,2015-01-14T17:32:49Z,...,True,True,7,7,203,135,1573082947,NaN,NaN,NaN
1,39256926,dadosgovbr/catalogos-dados-brasil,Mapeamento de iniciativas (e catálogos) de dad...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/catalogos-dados-...,https://api.github.com/repos/dadosgovbr/catalo...,False,2015-07-17T14:02:34Z,...,True,True,34,34,0,106,1573082947,NaN,NaN,NaN
2,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,False,2016-11-10T13:35:40Z,...,True,True,18,18,1,48,1573082947,NaN,NaN,NaN
3,49720381,dadosgovbr/aplicativos-dados-brasil,Mapeamento de aplicativos e visualizações que ...,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/aplicativos-dado...,https://api.github.com/repos/dadosgovbr/aplica...,False,2016-01-15T13:29:14Z,...,True,True,9,9,1,48,1573082947,NaN,NaN,NaN
4,20032900,dadosgovbr/kit,Kit de implementação do Plano de Dados Abertos,Organization,https://api.github.com/users/dadosgovbr,https://github.com/dadosgovbr,https://github.com/dadosgovbr/kit,https://api.github.com/repos/dadosgovbr/kit,False,2014-05-21T18:44:02Z,...,True,False,15,15,10,39,1573082947,NaN,NaN,NaN


In [132]:
file_path = '../data/extract_repositories_' + str(time.time()).split('.')[0] + '.csv'
file_path

'../data/extract_repositories_1573083022.csv'

In [133]:
resultados.to_csv(file_path, index=False)

Forma de pegar o Readme de um repositório

In [ ]:
repo = g.get_repo("freeCodeCamp")
contents = repo.get_contents("README.md")
print(contents.decoded_content)